In [1]:
import sys
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')
#!pip install tqdm #--user
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
#!pip install gensim #--user
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

The english-only version of the doc2vec model (increased performances!) 

In [2]:
supp_df=pd.read_csv("meetup.csv")

In [3]:
supp_df=supp_df[supp_df["category"].isna()!=True]

In [4]:
cat_list=list(supp_df.category.unique())
cat_list=[str(elem) for elem in cat_list]
cat_list.sort()
cat_list=[elem.replace("/", " ") for elem in cat_list]
"""
cat_list_old= cat_list[:]
cat_list=[]
for elem in cat_list_old:
    if " " in elem:
        els=elem.split(" ")
        cat_list.append(els)
    else:
        cat_list.append(elem)
"""
cat_list

['LGBT',
 'alternative lifestyle',
 'book clubs',
 'career business',
 'cars motorcycles',
 'community environment',
 'dancing',
 'education learning',
 'fashion beauty',
 'fine arts culture',
 'fitness',
 'food drink',
 'games',
 'health wellbeing',
 'hobbies crafts',
 'language ethnic identity',
 'movements politics',
 'movies film',
 'music',
 'new age spirituality',
 'outdoors adventure',
 'paranormal',
 'parents family',
 'pets animals',
 'photography',
 'religion beliefs',
 'sci-fi fantasy',
 'singles',
 'socializing',
 'sports recreation',
 'support',
 'tech',
 'writing']

In [5]:
df=pd.read_csv("processed_description_sine_error_Porter_no_bad_words.csv")
df=df[df["lang_ok"]=="english"]
df.head(3)

,Unnamed: 0,description,category_bin,lang_ok,processed_description
0,0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,27,english,food fellowship concert may us morn star churc...
1,1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan collabor peopl come whynot japan ...
2,2,<p>TENTATIVE<br></p>\n<p><br></p>\n<p>10.30 - ...,31,english,tent introduct orient session break network au...


In [6]:
df["category"]="NaN"

df["category"]= df.apply(lambda r: cat_list[r.category_bin], axis=1)
del supp_df
df.head(2)

,Unnamed: 0,description,category_bin,lang_ok,processed_description,category
0,0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,27,english,food fellowship concert may us morn star churc...,singles
1,1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan collabor peopl come whynot japan ...,socializing


In [7]:
process_all=True
#process_all=True

if process_all:
    train, test = train_test_split(df, test_size=0.3, random_state=42)
else:    
    train, test = train_test_split(df[:50000], test_size=0.3, random_state=42)

train_tagged = train.apply(
    lambda r: TaggedDocument(words=str(r.processed_description).split(" "), tags=[r.category]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=str(r.processed_description).split(" "), tags=[r.category]), axis=1)

In [8]:
#train_tagged.values[30]

In [9]:
import multiprocessing

#cores = multiprocessing.cpu_count()
#cores
cores=4

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)

In [10]:
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 114181/114181 [00:00<00:00, 3951596.82it/s]


In [11]:
%%time
for epoch in range(10):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 114181/114181 [00:00<00:00, 3531106.31it/s]


CPU times: user 6min 38s, sys: 6.14 s, total: 6min 45s
Wall time: 1min 56s


In [12]:
def vec_for_learning(model, tagged_docs):
    #sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in tqdm(tagged_docs.values)])
    return targets, regressors

In [13]:
%%time
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

100%|██████████| 48935/48935 [03:20<00:00, 244.52it/s]

CPU times: user 11min 35s, sys: 1.14 s, total: 11min 36s
Wall time: 11min 34s


In [30]:
#go=False
go=True
if go:
    import joblib
    joblib.dump(model_dbow, "model_dbow_all_only_eng_10_ep_Porter_no_bad_words.joblib")
    process_all=False
    if process_all:
        joblib.dump(X_train, "X_train_all_10_ep_Porter_no_bad_words.joblib")
        joblib.dump(y_train, "y_train_all_10_ep_Porter_no_bad_words.joblib")
        joblib.dump(X_test, "X_test_all_10_ep_Porter_no_bad_words.joblib")
        joblib.dump(y_test, "y_test_all_10_ep_Porter_no_bad_words.joblib")

In [19]:
go=False
#go=True

if go:
    logreg = LogisticRegression(n_jobs=5, C=1e5, solver="sag")

    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)

    from sklearn.metrics import accuracy_score, f1_score

    print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
    print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.6514178541577478
Testing F1 score: 0.6497767054978865


In [18]:
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 114181/114181 [00:00<00:00, 2170932.25it/s]


In [19]:
%%time
for epoch in range(10):
    model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

100%|██████████| 114181/114181 [00:00<00:00, 2937268.16it/s]


CPU times: user 9min 46s, sys: 13.6 s, total: 9min 59s
Wall time: 2min 35s


In [20]:
y_train_dmm, X_train_dmm = vec_for_learning(model_dmm, train_tagged)
y_test_dmm, X_test_dmm = vec_for_learning(model_dmm, test_tagged)


100%|██████████| 48935/48935 [04:53<00:00, 166.94it/s]


In [29]:
go=True
#go=True
if go:
    import joblib
    joblib.dump(model_dmm, "model_ddm_all_only_eng_10_ep_Porter_no_badwords.joblib")
    process_all=False
    if process_all:
        joblib.dump(X_train_dmm, "X_train_all_ddm_10_ep_Porter_no_badwords.joblib")
        joblib.dump(y_train_dmm, "y_train_all_ddm_10_ep_Porter_no_badwords.joblib")
        joblib.dump(X_test_dmm, "X_test_all_ddm_10_ep_Porter_no_badwords.joblib")
        joblib.dump(y_test_dmm, "y_test_all_ddm_10_ep_Porter_no_badwords.joblib")

In [31]:
model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [32]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

In [33]:
y_train_pair, X_train_pair = vec_for_learning(new_model, train_tagged)
y_test_pair, X_test_pair = vec_for_learning(new_model, test_tagged)

100%|██████████| 48935/48935 [08:37<00:00, 94.60it/s] 


In [34]:
go=True
if go:
    import joblib
    joblib.dump(new_model, "model_pair_all_only_eng_10_ep_Porter_no_badwords.joblib")
    process_all=False
    if process_all:
        joblib.dump(X_train_pair, "X_train_all_pair_10_ep_Porter_no_badwords.joblib")
        joblib.dump(y_train_pair, "y_train_all_pair_10_ep_Porter_no_badwords.joblib")
        joblib.dump(X_test_pair, "X_test_all_pair_10_ep_Porter_no_badwords.joblib")
        joblib.dump(y_test_pair, "y_test_all_pair_10_ep_Porter_no_badwords.joblib")

In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, roc_curve, auc

In [15]:
forest = RandomForestClassifier(n_jobs=4, n_estimators=100) 
forest = forest.fit(X_train, y_train)

In [26]:
save=True
if save:
    import joblib
    joblib.dump(forest, "model_rf_only_eng_dbow_72acc.joblib")

In [17]:
print(forest.score(X_test, y_test))

predictions = forest.predict(X_test)

print(metrics.classification_report(y_test, predictions)) #100 features

0.7247573311535711
                          precision    recall  f1-score   support

                    LGBT       0.99      0.35      0.52       463
   alternative lifestyle       0.97      0.78      0.86       223
              book clubs       0.68      0.45      0.54       388
         career business       0.70      0.83      0.76      4580
        cars motorcycles       0.98      0.50      0.66       362
   community environment       0.82      0.46      0.59       944
                 dancing       0.82      0.81      0.82      1090
      education learning       0.79      0.48      0.60      1003
          fashion beauty       0.71      0.40      0.51        81
       fine arts culture       0.82      0.60      0.69      1027
                 fitness       0.79      0.69      0.74      2269
              food drink       0.75      0.56      0.64      1424
                   games       0.85      0.82      0.83      1469
        health wellbeing       0.71      0.81      0.75 

In [23]:
forest1 = RandomForestClassifier(n_jobs=6, n_estimators=200) 
forest1 = forest.fit(X_train_dmm, y_train_dmm)

In [24]:
forest1.score(X_test_dmm, y_test_dmm)

0.5346071319096761

In [25]:
save=True
if save:
    joblib.dump(forest1, "model_rf_only_eng_dmm.joblib")

In [27]:
predictions = forest1.predict(X_test_dmm)
print(metrics.classification_report(y_test_dmm, predictions)) #100 features

                          precision    recall  f1-score   support

                    LGBT       1.00      0.14      0.25       463
   alternative lifestyle       0.66      0.67      0.67       223
              book clubs       0.96      0.11      0.20       388
         career business       0.46      0.75      0.57      4580
        cars motorcycles       0.98      0.17      0.29       362
   community environment       0.94      0.17      0.29       944
                 dancing       0.86      0.44      0.58      1090
      education learning       0.84      0.25      0.38      1003
          fashion beauty       0.73      0.14      0.23        81
       fine arts culture       0.92      0.29      0.44      1027
                 fitness       0.68      0.44      0.53      2269
              food drink       0.82      0.25      0.38      1424
                   games       0.92      0.37      0.53      1469
        health wellbeing       0.47      0.70      0.56      4532
         

In [35]:
forest_pair = RandomForestClassifier(n_jobs=6) 
forest_pair = forest.fit(X_train_pair, y_train_pair)

In [36]:
forest_pair.score(X_test_pair, y_test_pair)#100feat

0.7161949524879943

In [37]:
save=True
if save:
    joblib.dump(forest_pair, "model_rf_pair_only_eng.joblib")

In [38]:
predictions = forest_pair.predict(X_test_pair)
print(metrics.classification_report(y_test_pair, predictions)) #100 features

                          precision    recall  f1-score   support

                    LGBT       0.99      0.33      0.49       463
   alternative lifestyle       0.97      0.75      0.85       223
              book clubs       0.71      0.44      0.54       388
         career business       0.68      0.84      0.75      4580
        cars motorcycles       0.98      0.46      0.63       362
   community environment       0.83      0.45      0.58       944
                 dancing       0.82      0.80      0.81      1090
      education learning       0.80      0.46      0.59      1003
          fashion beauty       0.76      0.31      0.44        81
       fine arts culture       0.82      0.59      0.68      1027
                 fitness       0.80      0.69      0.74      2269
              food drink       0.75      0.55      0.63      1424
                   games       0.85      0.80      0.82      1469
        health wellbeing       0.70      0.80      0.75      4532
         